In [44]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%%capture
!pip install transformers
!pip install evaluate

In [1]:
import pandas as pd
import datasets
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer

# Load the dataset
train_df = pd.read_csv('./NLP Dataset/final_train.csv')
test_df = pd.read_csv('./NLP Dataset/final_test.csv')

# Rename the columns to match the expected column names
train_df = train_df.rename(columns={'unprocessed_text': 'text', 'class': 'label'})
test_df = test_df.rename(columns={'unprocessed_text': 'text', 'class': 'label'})

#remove tweeetid
train_df = train_df.drop('tweetid',axis=1,)
train_df = test_df.drop('tweetid',axis=1,)


In [2]:
train_df.head(25)

,text,label
0,she sprinkled adderall on her granola with ric...,m
1,vyvanse had me telling my life story to a 50 y...,c
2,@Elianok10 @letthembekids2 @invisable22 @kpana...,c
3,@lisarinna Now that i've got xanax in my smoot...,a
4,adderall had me doing sit ups at midnight,c
5,"i take cymbalta, gabapentin, morphine, flexeri...",c
6,"why does everyone want adderall today man, shi...",m
7,No Mattah what that hoe said or wanna switch u...,m
8,@melistittsss i got you with a xanax if you ne...,a
9,Have blurry vision?You probably have Autophili...,m


In [3]:
# Convert the labels to integers (m=2, c=1, a=0, u=3)
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

In [4]:
print(train_df['text'].dtype)     # Check data type of 'text' column in train_df
print(train_df['text'].shape)     # Check shape of 'text' column in train_df
print(test_df['text'].dtype)      # Check data type of 'text' column in test_df
print(test_df['text'].shape)      # Check shape of 'text' column in test_df
print(train_df['text'])
train_df.head(25)

object
(3271,)
object
(3271,)
0       she sprinkled adderall on her granola with ric...
1       vyvanse had me telling my life story to a 50 y...
2       @Elianok10 @letthembekids2 @invisable22 @kpana...
3       @lisarinna Now that i've got xanax in my smoot...
4               adderall had me doing sit ups at midnight
                              ...                        
3266    See that girl who definitely WASN'T smuggling ...
3267    that one time i took adderall and tried to wri...
3268    Here's to me taking penicillin thinking it was...
3269    @manofletterspdx @hotspurjp I bet I'd have a h...
3270    @UKinUSA hi. Can you possibly advise. Can I br...
Name: text, Length: 3271, dtype: object


,text,label
0,she sprinkled adderall on her granola with ric...,2
1,vyvanse had me telling my life story to a 50 y...,1
2,@Elianok10 @letthembekids2 @invisable22 @kpana...,1
3,@lisarinna Now that i've got xanax in my smoot...,0
4,adderall had me doing sit ups at midnight,1
5,"i take cymbalta, gabapentin, morphine, flexeri...",1
6,"why does everyone want adderall today man, shi...",2
7,No Mattah what that hoe said or wanna switch u...,2
8,@melistittsss i got you with a xanax if you ne...,0
9,Have blurry vision?You probably have Autophili...,2


In [5]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=4)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
# Tokenize the dataset
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

tokenized_train_dataset = datasets.Dataset.from_pandas(train_df)
tokenized_train_dataset = tokenized_train_dataset.map(tokenize, batched=True)

tokenized_test_dataset = datasets.Dataset.from_pandas(test_df)
tokenized_test_dataset = tokenized_test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3271 [00:00<?, ? examples/s]

Map:   0%|          | 0/3271 [00:00<?, ? examples/s]

In [7]:
tokenized_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3271
})

In [8]:
print(tokenized_train_dataset['text'])
print(tokenized_train_dataset['label'])

['she sprinkled adderall on her granola with rice milk and headed off to bomb her 8 am final.', 'vyvanse had me telling my life story to a 50 year old korean man at the hyundai dealership.', "@Elianok10 @letthembekids2 @invisable22 @kpanarella @bitterexdroid ONLY the ones I take now helped - MORPHINE didn't even help.", "@lisarinna Now that i've got xanax in my smoothie im as cool as a cucumber, want a sip? It's vegan!", 'adderall had me doing sit ups at midnight', 'i take cymbalta, gabapentin, morphine, flexeril, adderall (only am or workday/not daily) baby aspirin &amp; one of them gives me vivid dreams', 'why does everyone want adderall today man, shit is annoying😒', 'No Mattah what that hoe said or wanna switch up n say, I know animals get tranquilized n I know "OLANZAPINE"is "TRANQUILIZER"', '@melistittsss i got you with a xanax if you need it.', 'Have blurry vision?You probably have Autophilia.This causes eye hemorrhagingTry Xanax', 'Smackheads sat in A&amp;E arguing about heroin

In [9]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results2',          # Output directory
    evaluation_strategy = "epoch",   # Evaluation strategy
    # learning_rate=2e-5,              # Learning rate
    # per_device_train_batch_size=16,  # Batch size
    # num_train_epochs=3,              # Number of training epochs
    # weight_decay=0.01,               # Weight decay
    # push_to_hub=False,
)

In [10]:
# Define the trainer
trainer = Trainer(
    model=model,                         # The model to be trained
    args=training_args,                  # Training arguments
    train_dataset=tokenized_train_dataset, # Training dataset
    eval_dataset=tokenized_test_dataset,  # Evaluation dataset
)

In [11]:
# Train the model
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3271
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1227
  Number of trainable parameters = 108313348


  0%|          | 0/1227 [00:00<?, ?it/s]

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate(tokenized_test_dataset)
print(eval_results)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, tweetid. If text, tweetid are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3271
  Batch size = 8


  0%|          | 0/409 [00:00<?, ?it/s]

{'eval_loss': 0.10606889426708221, 'eval_runtime': 561.5762, 'eval_samples_per_second': 5.825, 'eval_steps_per_second': 0.728, 'epoch': 3.0}


In [ ]:
#calculate overall f1 score

import numpy as np

# Make predictions on the test set
predictions, _, _ = trainer.predict(tokenized_test_dataset)

# Convert predicted probabilities to predicted classes
predicted_classes = np.argmax(predictions, axis=1)

# Calculate F1 score
from sklearn.metrics import f1_score
f1 = f1_score(test_df['label'], predicted_classes, average='macro')
print("F1 score:", f1)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, tweetid. If text, tweetid are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3271
  Batch size = 8


  0%|          | 0/409 [00:00<?, ?it/s]

F1 score: 0.9639356234348367


In [ ]:
# Calculate F1 score for label 0 (a)
f1 = f1_score(test_text['label'], predicted_classes, labels=[0], average='macro')
print("F1 score for label 0:", f1)